In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from random import randint
from time import sleep

In [19]:
chrome_driver_path = "C:/Users/austi/Documents/Github_Repos/Imperial_Applied_Project/ChromeDriver/chromedriver.exe"
chrome_profile_path = r'user-data-dir=C:\Users\austi\AppData\Local\Google\Chrome\User Data\Default'
options = webdriver.ChromeOptions()
options.add_argument(chrome_profile_path)
options.add_argument("--start-maximized")
options.add_argument("--disable-popup-blocking")
options.add_argument("--no-sandbox")
options.add_argument("--disable-impl-side-painting")
options.add_argument("--disable-setuid-sandbox")
options.add_argument("--disable-seccomp-filter-sandbox")
options.add_argument("--disable-single-click-autofill")

driver = webdriver.Chrome(executable_path=chrome_driver_path, options=options)

In [11]:
#listing_button = driver.find_element_by_id('listing-type').click()
#--disable-autofill-keyboard-accessory-view
#--disable-physical-keyboard-autocorrect
#--disable-single-click-autofill
#48126
#48201

In [8]:
search_box = driver.find_element_by_css_selector('input.react-autosuggest__input')

In [17]:
search_box.clear()

In [18]:
search_box.click()

In [15]:
count = 0
while count < 4:
    search_box.send_keys(Keys.BACKSPACE)
    count+= 1

In [13]:
search_box.send_keys('48126')

In [43]:
search_box.send_keys(Keys.ENTER)

In [9]:
try:
    search_box = WebDriverWait(driver, 4).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input.react-autosuggest__input')))
    search_box.clear()
finally:
    driver.quit()
search_box.send_keys('Detroit')
driver.implicitly_wait(1)
search_box.send_keys(Keys.RETURN)

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=53396): Max retries exceeded with url: /session/f3ce3b45cf02ca6fc1cc2c305e210aeb/element/7df4e6cf-d7cf-4060-8e8c-04714dcb5569/value (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001AF392406D8>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it',))

In [62]:
photo_cards = driver.find_element_by_css_selector('ul.photo-cards.photo-cards_wow')

In [66]:
for listing in photo_cards.find_elements_by_tag_name("li"):
    if listing.get_attribute('class') == 'nav-ad-empty':
        continue
    house = listing.click()
    house.find_element_by_css_selector('div.character-count-text-fold-container').get_attribute('div')
    sleep(randint(1,3))
    house.find_element_by_css_selector('button.ds-close-lightbox-icon.hc-back-to-list').click()

AttributeError: 'NoneType' object has no attribute 'find_element_by_css_selector'

In [46]:
<div class="character-count-text-fold-container"><div class="" style="white-space: pre-wrap; font-size: 15px; line-height: 1.5;">VERY NICE 4 BEDROOM HOME IN EXCELLENT NEIGHBORHOOD. HOME HAS AN OPEN LAYOUT AND WITH A LITTLE TLC IT COULD BE SPECTACULAR.</div></div>

In [48]:
driver.find_element_by_css_selector('button.ds-close-lightbox-icon.hc-back-to-list').click()
#<button class="ds-close-lightbox-icon hc-back-to-list" aria-label="close button"><svg width="24px" height="24px" xmlns="http://www.w3.org/2000/svg"><path d="M11.778 11.778L4 4l7.778 7.778L4 19.556l7.778-7.778zm0 0l7.778 7.778-7.778-7.778L19.556 4l-7.778 7.778z" stroke="#FFF" stroke-width="2" fill="none" fill-rule="evenodd" stroke-linecap="round" stroke-linejoin="round"></path></svg></button>

In [50]:
for listing in driver.find_element_by_css_selector('ul.photo-cards.photo-cards_wow'):
    house = driver.find_element_by_tag_name("li").text

TypeError: 'WebElement' object is not iterable

In [ ]:
search_box.send_keys(Keys.RETURN)

In [72]:
photo_cards = driver.find_element_by_css_selector('ul.photo-cards.photo-cards_wow')
house = photo_cards.find_element_by_tag_name("li").click()


In [73]:
driver.close()

WebDriverException: Message: chrome not reachable
  (Session info: chrome=75.0.3770.100)
